In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

task = 0
mlp_layers = 4
mlp_dim = 768 #768
max_epochs = 2000
lr = 0.01
min_lr = 0.0001

# T-matrix data
filename = '/Dedicated/jwang-data/xchen/non-spherical/NN/py/results/tmat_fine_n_intg.pickle'
with open(filename, 'rb') as f:
    TMdata = pickle.load(f)
X_ori = np.reshape(np.array(TMdata['x']),(int(len(TMdata['x'])/4),4))
y_ori = np.log(TMdata['value']['ext'])
xk    = X_ori[:,2]
xn    = X_ori[:,3]
J_ori = np.reshape( np.append( TMdata['Jacmi']['ext'] / X_ori[:,2], TMdata['Jacmr']['ext'] ), (2,len(TMdata['Jacmr']['ext'])) )
idx   = np.where( (np.abs(X_ori[:,-1]-1.291429) > 1.0e-6 ) & (np.abs(X_ori[:,-1]-1.33) > 1.0e-6 ) )[0]
X1    = X_ori[idx,:]
X1[:,1] = np.log(X1[:,1])
X1[:,2] = np.log(X1[:,2])
y1    = y_ori[idx].reshape(-1,1)
J1    = J_ori[:,idx].T
J2    = np.empty(J1.shape)
J2[:,0]  = J1[:,0] / np.exp(y1[:,0]) * xk[idx]
J2[:,1]  = J1[:,1] / np.exp(y1[:,0]) * xn[idx]

print( X_ori[100,:], J1[:10,:], J2[:10,:], X1[::181*14*5,-1], X1[100,:], y1.shape )

# combine
J_mean = J2.mean(axis=0, keepdims=True)
J_std  = J2.std(axis=0, keepdims=True)
norm_J = (J2 - J_mean) / J_std
num_tasks = 1
allX  = X1
ally  = y1
allJ  = J1 #norm_J
UseX  = allX
num_examples, num_features = UseX.shape
Usey  = np.hstack((ally,allJ))
    
#
#
print(num_examples, Usey.shape)
print(X1[100,:],y1, Usey[-10:,:])
    



[1.00000000e+02 8.50350014e-04 5.00000024e-04 1.29142904e+00] [[ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]
 [ 8.80761984e-03 -2.30353642e-06]] [[ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]
 [ 0.99967602 -0.67754029]] [1.29571462 1.30000031 1.304286   1.3085717  1.31285739 1.31714296
 1.32142866 1.32571435] [100.          -7.06986251  -7.60090241   1.29571462] (101360, 1)
101360 (101360, 3)
[100.          -7.06986251  -7.60090241   1.29571462] [[-12.33271642]
 [-12.33271642]
 [-12.33271642]
 ...
 [ -6.31135514]
 [ -6.31135514]
 

In [2]:
# Split dataset
train_x, test_x, train_y, test_y = train_test_split(UseX, Usey, test_size=0.2)
    
x_mean = train_x.mean(axis=0, keepdims=True)
x_std = train_x.std(axis=0, keepdims=True)
train_x = (train_x - x_mean) / x_std
test_x = (test_x - x_mean) / x_std
    
train_x = torch.Tensor(train_x)
train_y = torch.Tensor(train_y)
test_x = torch.Tensor(test_x)
test_y = torch.Tensor(test_y)
    
# Transform data
x_mean = torch.Tensor(x_mean).cuda()
x_std = torch.Tensor(x_std).cuda()
J_mean = torch.Tensor(J_mean).cuda()
J_std = torch.Tensor(J_std).cuda()

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

for sf in [1.0]: #[0.1, 0.01, 0.001]:
        
    # Construct Pytorch/Skorch model
    class AttrProxy(object):
        """Translates index lookups into attribute lookups."""
        def __init__(self, module, prefix):
            self.module = module
            self.prefix = prefix
    
        def __getitem__(self, i):
            return getattr(self.module, self.prefix + str(i))
  
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
    
            input_dim = num_features
            for i in range(mlp_layers-1):
                self.add_module('fc_' + str(i), nn.Linear(input_dim, mlp_dim))
                input_dim = mlp_dim
            self.fc = AttrProxy(self, 'fc_')
            self.fc1 = nn.Linear(input_dim, num_tasks)
    
        def forward(self, x):
            for i in range(mlp_layers-1):
                fc = self.fc.__getitem__(i)
                x = torch.tanh(fc(x))
            x = self.fc1(x)
            return x

    from skorch.utils import TeeGenerator
    class MyNeuralNetRegressor(NeuralNetRegressor):
        def train_step_single(self, Xi, yi, **fit_params):
            Xi = Xi.requires_grad_(True)
            batch_size = Xi.shape[0]
            
            # split yi into values and jacobians
            y0  = torch.reshape(yi[:,0],(len(yi[:,0]),1))
            self.module_.train()
            y_pred = self.infer(Xi, **fit_params)
            #print('y_pred',y_pred)
            #print('y0',y0)
        
            y_pred.backward(torch.eye(batch_size,).to(device), retain_graph=True)
            
            # gradient calculation
            self.module_.zero_grad()
            grad = Xi.grad[:,2:]
            grad_ref = yi[:,1:].requires_grad_(True)       
            grad0 = grad[:,0] / x_std[0][2] 
            grad1 = grad[:,1] / x_std[0][3]
            normgrad0 = (grad0 - J_mean[0][0]) / J_std[0][0]
            normgrad1 = (grad1 - J_mean[0][1]) / J_std[0][1]
            
            # gradient loss
            #print('J1_pred',J1_pred, grad_ref[:,1] * sample_weight)
            grad_loss = self.get_loss(normgrad0, grad_ref[:,0], X=Xi, training=True) + \
                        self.get_loss(normgrad1, grad_ref[:,1], X=Xi, training=True)
            #print('grad_loss',grad_loss)
            #loss = self.get_loss(y_pred, y0, X=Xi, training=True) + \
            #                s1 * delta_grad0 + s2 * delta_grad1
            
            loss = self.get_loss(y_pred, y0, X=Xi, training=True)  + grad_loss
                            
            loss.backward()
            self.notify(
                    'on_grad_computed',
                    named_parameters=TeeGenerator(self.module_.named_parameters()),
                    X=Xi,
                    y=y0
                )

            return {
                    'loss': loss,
                    'y_pred': y_pred,
                    }
    
    
        def validation_step(self, Xi, yi, **fit_params):
            """Perform a forward step using batched data and return the
                resulting loss.
                The module is set to be in evaluation mode (e.g. dropout is
                not applied).
                Parameters
                ----------
                Xi : input data
                  A batch of the input data.
                yi : target data
                  A batch of the target data.
                **fit_params : dict
                  Additional parameters passed to the ``forward`` method of
                  the module and to the ``self.train_split`` call.
            """
                
            # split yi
            y0  = torch.reshape(yi[:,0],(len(yi[:,0]),1))
            Xi = Xi.requires_grad_(True)
            batch_size = Xi.shape[0]
                
            self.module_.eval()
            y_pred = self.infer(Xi, **fit_params)
            y_pred.backward(torch.eye(batch_size,).to(device), retain_graph=True)
                
            grad = Xi.grad[:,2:]
            grad_ref = yi[:,1:]
                
            grad0 = grad[:,0] / x_std[0][2] 
            grad1 = grad[:,1] / x_std[0][3]
            normgrad0 = (grad0 - J_mean[0][0]) / J_std[0][0]
            normgrad1 = (grad1 - J_mean[0][1]) / J_std[0][1]
            
            grad_loss = self.get_loss(normgrad0, grad_ref[:,0], X=Xi, training=False) + \
                        self.get_loss(normgrad1, grad_ref[:,1], X=Xi, training=False)
            #print('valid grad:', grad)
                
            yloss = self.get_loss(y_pred, y0, X=Xi, training=False)
            loss = yloss + grad_loss
            y_loss.append( yloss.item() )
            allgrad_loss.append( grad_loss.item() )

            #print('valid_loss:', grad_loss, yloss)

#             y_loss.append(yloss.item())
#             grad0_loss.append( s1 * delta_grad0.item() )
#             grad1_loss.append( s2 * delta_grad1.item() )
                
            return {
                    'loss': loss,
                    'y_pred': y_pred,
                    }


    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    net_regr = MyNeuralNetRegressor(
            Net,
            max_epochs=max_epochs,
            batch_size=1024,
            lr=lr,
            device=device,
    )

    from skorch.callbacks import Checkpoint, LoadInitState, LRScheduler
    cp1 = Checkpoint(dirname='./tanh_logX_n_add_oriJ_10sf/')  #tanh_logX_n_add_lnJ

    net_regr.initialize()
    net_regr.load_params(checkpoint=cp1)

    train_score = net_regr.score(train_x.cpu(), train_y[:,0].cpu())
    test_score = net_regr.score(test_x.cpu(), test_y[:,0].cpu())
    print(train_score, test_score)


0.9996978488791617 0.9996991813552724


In [7]:
rr0 = np.sort( np.array( list(set(list(np.exp(X0[:,1])))) ) )
rr1 = np.sort( np.array( list(set(list(np.exp(X1[:,1])))) ) )
kk0 = np.sort( np.array( list(set(list(np.exp(X0[:,2])))) ) )
kk1 = np.sort( np.array( list(set(list(np.exp(X1[:,2])))) ) )
nn0 = np.sort( np.array( list(set(list(X0[:,3]))) ) )
nn1 = np.sort( np.array( list(set(list(X1[:,3]))) ) )
print(rr0, rr1, kk0, kk1, nn0, nn1)

[6.480963e-04 8.503500e-04 1.115722e-03 1.463909e-03 1.920756e-03
 2.520173e-03 3.306652e-03 4.338570e-03 5.692523e-03 7.469008e-03
 9.799887e-03 1.285817e-02 1.687087e-02 2.213582e-02 2.904382e-02
 3.810762e-02 5.000000e-02 6.560367e-02 8.607684e-02 1.129391e-01
 1.481844e-01 1.944289e-01 2.551050e-01 3.347165e-01 4.391726e-01
 5.762267e-01 7.560518e-01 9.919955e-01 1.301571e+00 1.707757e+00
 2.240702e+00 2.939966e+00 3.857452e+00 5.061260e+00 6.640745e+00
 8.713145e+00 1.143229e+01 1.500000e+01 1.968110e+01 2.582305e+01
 3.388174e+01] [0.00085035 0.00111572 0.00146391 0.00192076 0.00252017 0.00330665
 0.00433857 0.00569252 0.00746901 0.00979989 0.01285817 0.01687087
 0.02213582 0.02904382] [0.0005     0.00081895 0.00134135 0.00219698 0.00359843 0.00589384
 0.00965349 0.01581139 0.02589737 0.04241714 0.06947477 0.1137923
 0.1863797  0.3052701  0.5       ] [0.0005     0.00219699 0.00965349 0.04241714 0.1863797 ] [1.291429 1.310714 1.33     1.349286 1.368571 1.387857 1.407143 1.426429
 

In [4]:
# Dubovik's database
X0 = np.load("/Dedicated/jwang-data/xchen/non-spherical/NN/py/results/X_untrans.npy")
X0[:,1] = np.log(X0[:,1])
X0[:,2] = np.log(X0[:,2])
y0 = np.log(np.load("/Dedicated/jwang-data/xchen/non-spherical/NN/py/results/y_untrans.npy")[:, task:task+1])


# read Tmatrix data
#X0   = X
flag = np.logical_and(X0[:,1] > np.log(0.0008),X0[:,1] < np.log(0.03))
idx0 = np.where(flag == True)[0]
pred_X = X0[idx0,:]
print(pred_X.shape)

tmaty = {}
tmatJ = {}
with open('/Dedicated/jwang-data/xchen/non-spherical/NN/py/results/tmat_Jacall_norm_intg.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
for key in data2['value'].keys():
    tmaty[key] = data2['value'][key]
    tmatJ[key] = np.reshape( np.append( data2['Jacmi'][key], data2['Jacmr'][key] ), (2,len(data2['value'][key])) )
cho_idx = data2['idx']
print('cho_idx',len(cho_idx))

(836220, 4)
cho_idx 101360


In [6]:
Xa = X0
y0 = y
def get_ut_jacobian(net, x):
    batch_size = x.shape[0]
    xt = x 
    xt.requires_grad_(True)
    y = torch.exp(net(xt))
    #print(y.detach().cpu().numpy().shape)
    y.backward(torch.eye(batch_size).cuda())
    return y.detach().cpu().numpy(), xt.grad.detach().cpu().numpy(), xt.detach().cpu().numpy()

def get_jacobian(net, x):
    batch_size = x.shape[0]
    x.requires_grad_(True)
    y = net(x)
    y.backward(torch.eye(batch_size).cuda())
    return y.detach().cpu().numpy(), x.grad.detach().cpu().numpy()

def make_plot_lnx2(n=1):
    
    
    x2 = np.linspace(Xa[0, 2], Xa[(n-1)*41*181, 2], 500)
    x0 = np.ones_like(x2)*Xa[0, 0]
    x1 = np.ones_like(x2)*Xa[182, 1]
    x3 = np.ones_like(x2)*Xa[0, 3]
    Xp = np.stack([x0, x1, x2, x3], axis=1)
    print(Xp.shape, Xp[5,:],pred_X[cho_idx,:][0:181*14*5:181*14,:])
    Xp = torch.Tensor(Xp).cuda()
    Xp = (Xp-x_mean)/x_std
    yp, jac, xt = get_ut_jacobian(net_regr.module_, Xp)
    print('jac:',jac.shape)
    #-x_mean.cpu().numpy())/x_std.cpu().numpy()
    fig, ax1 = plt.subplots()
    Xp = Xp.detach().cpu().numpy()
    print(x_std.cpu().numpy())
    ax1.scatter(x2, jac[:, 2]/x_std.cpu().numpy()[0][2], s=1, c='C0', alpha=0.7)
    ax1.scatter(pred_X[cho_idx,2][0:181*14*5:181*14], tmatJ['ext'][0,0:181*14*5:181*14], s=5, c='C3', alpha=0.7 )
    ax1.set_ylabel(r'$\frac{\partial{cext}}{\partial{lnk}}$', color='C0')
    ax1.tick_params(axis='y', labelcolor='C0')
    ax1.set_xlabel('lnk')
    ax1.legend(['NN-Jac','Tmat-Jac'],loc='upper left')
    ax2 = ax1.twinx()
    ax2.scatter(x2, yp, s=1, c='C1', alpha=0.7, label='predict')
    for ip in range(n):
        xl = Xa[181*41*ip,:] 
        if ip == 0:
            ax2.scatter(xl[2], np.exp(y0[181*41*ip]), s=5, c='C2', alpha=0.7, label='true')
        else:
            ax2.scatter(xl[2], np.exp(y0[181*41*ip]), s=5, c='C2', alpha=0.7)
    ax2.scatter(pred_X[cho_idx,2][0:181*14*5:181*14], tmaty['ext'][0:181*14*5:181*14], s=5, c='C4', alpha=0.7, label='Tmat')
    ax2.legend(loc='lower right')
    ax2.set_ylabel('cext', color='C1')
    ax2.tick_params(axis='y', labelcolor='C1')
    plt.tight_layout()
    plt.savefig('img/jac_logX_lnk_tanh_base_lnJ_fine_n.png', dpi=300, facecolor='w', edgecolor='w')
    plt.close()
    
def make_plot_x2(n=1):
    
    
    x2 = np.linspace(Xa[0,2], Xa[(n-1)*41*181, 2], 1000)
    x0 = np.ones_like(x2)*Xa[0, 0]
    x1 = np.ones_like(x2)*Xa[182, 1]
    x3 = np.ones_like(x2)*Xa[0, 3]
    Xp = np.stack([x0, x1, x2, x3], axis=1)
    print(Xp.shape)
    Xp = torch.Tensor(Xp).cuda()
    Xp = (Xp-x_mean)/x_std
    yp, jac, xt = get_ut_jacobian(net_regr.module_, Xp)
    print('jac:',jac.shape)
    #-x_mean.cpu().numpy())/x_std.cpu().numpy()
    Xp = Xp.detach().cpu().numpy()
    print(x_std.cpu().numpy())
    fig, ax1 = plt.subplots()
    ax1.scatter(x2, jac[:, 2]/x_std.cpu().numpy()[0][2]/np.exp(x2), s=1, c='C0', alpha=0.7)
    tJ  = tmatJ['ext'][0,:]/np.exp(pred_X[cho_idx,2])
    ax1.scatter(pred_X[cho_idx,2][0:181*14*5:181*14], tJ[0:181*14*5:181*14], s=5, c='C3', alpha=0.7 )
    ax1.set_ylabel(r'$\frac{\partial{cext}}{\partial{k}}$', color='C0')
    ax1.tick_params(axis='y', labelcolor='C0')
    ax1.set_xlabel('k')
    ax1.legend(['NN-Jac','Tmat-Jac'],loc='upper left')
    ax2 = ax1.twinx()
    ax2.scatter(x2, yp, s=1, c='C1', alpha=0.7, label='predict')
    for ip in range(n):
        xl = Xa[181*41*ip,:] 
        if ip == 0:
            ax2.scatter(xl[2], np.exp(y0[181*41*ip]), s=5, c='C2', alpha=0.7, label='true')
        else:
            ax2.scatter(xl[2], np.exp(y0[181*41*ip]), s=5, c='C2', alpha=0.7)
    ax2.scatter(pred_X[cho_idx,2][0:181*14*5:181*14], tmaty['ext'][0:181*14*5:181*14], s=5, c='C4', alpha=0.7, label='Tmat')
    ax2.legend(loc='lower right')
    ax2.set_ylabel('cext', color='C1')
    ax2.tick_params(axis='y', labelcolor='C1')
    plt.tight_layout()
    plt.savefig('img/jac_logX_k_tanh_base_lnJ_fine_n.png', dpi=300, facecolor='w', edgecolor='w')
    plt.close()
    
def make_plot_x3(n=1):
    
    x3 = np.linspace(Xa[0, 3], Xa[(n-1)*41*181*15, 3], 1000)
    x2 = np.ones_like(x3)*Xa[0, 2]
    x0 = np.ones_like(x3)*Xa[0, 0]
    x1 = np.ones_like(x3)*Xa[182, 1]
    Xp = np.stack([x0, x1, x2, x3], axis=1)
    print(Xp.shape, Xp[5,:], pred_X[cho_idx,:][::181*14*5,:], X1[::181*14*5,:])
    Xp = torch.Tensor(Xp).cuda()
    Xp = (Xp-x_mean)/x_std
    yp, jac, xt = get_ut_jacobian(net_regr.module_, Xp)
    print('jac:',jac.shape)
    Xp = Xp.detach().cpu().numpy()
    print(x_std.cpu().numpy())
    fig, ax1 = plt.subplots()
    ax1.scatter(x3, jac[:, 3]/x_std.cpu().numpy()[0][3], s=1, c='C0', alpha=0.7)
    ax1.scatter(pred_X[cho_idx,3][::181*14*5], tmatJ['ext'][1,::181*14*5], s=5, c='C3', alpha=0.7 )
    ax1.scatter(X1[::181*14*5,3], J1[::181*14*5,1], s=5, c='C3', alpha=0.7 )
    ax1.set_ylabel(r'$\frac{\partial{cext}}{\partial{n}}$', color='C0')
    ax1.tick_params(axis='y', labelcolor='C0')
    ax1.set_xlabel('n')
    ax1.legend(['NN-Jac','Tmat-Jac'],loc='upper right')
    ax2 = ax1.twinx()
    ax2.scatter(x3, yp, s=1, c='C1', alpha=0.7, label='predict')
    for ip in range(n):
        xl = Xa[181*41*15*ip,3] 
        if ip == 0:
            ax2.scatter(xl, np.exp(y0[181*41*15*ip]), s=5, c='C2', alpha=0.7,label='true')
        else:
            ax2.scatter(xl, np.exp(y0[181*41*15*ip]), s=5, c='C2', alpha=0.7)
    ax2.scatter(pred_X[cho_idx,3][::181*14*5], tmaty['ext'][::181*14*5], s=5, c='C4', alpha=0.7, label='Tmat')
    ax2.scatter(X1[::181*14*5,3], np.exp(y1[::181*14*5,0]), s=5, c='C4', alpha=0.7)
    ax2.legend(loc='lower right')
    ax2.set_ylabel('cext', color='C1')
    ax2.tick_params(axis='y', labelcolor='C1')
    plt.tight_layout()
    plt.savefig('img/jac_logX_n_tanh_base_lnJ_fine_n.png', dpi=300, facecolor='w', edgecolor='w')
    plt.close()
    

    
def make_ut_plot(n=1):
    
    x0 = np.linspace(X[41*(n-1), 0], X[41*(n)-1, 0], 500)
    x1 = np.ones_like(x0)*X[41*(n-1), 1]
    x2 = np.ones_like(x0)*X[41*(n-1), 2]
    Xp = np.stack([x0, x1, x2], axis=1)
    print(Xp.shape)
    Xp = torch.Tensor(Xp).cuda()

    yp, jac, xt = get_ut_jacobian(net_regr.module_, Xp)
    xl = (X[41*(n-1):41*n,:]-x_mean.cpu().numpy())/x_std.cpu().numpy()
    plt.scatter(xt[:, 0], jac[:, 0])
    plt.scatter(xt[:, 0], yp)
    plt.scatter(xl[:, 0], np.exp(y[41*(n-1):41*n]))
    plt.legend(['jac', 'y', 'target'])
    plt.savefig('jac_ut.png', dpi=300)
    plt.close()

#make_ut_plot(n=1)
make_plot_lnx2(n=15)

#fig = plt.figure()
make_plot_x2(n=15)

make_plot_x3(n=22)

(500, 4) [ 0.         -7.06986253 -7.53168647  1.291429  ] [[ 0.         -7.06986253 -7.60090246  1.291429  ]
 [ 0.         -7.06986253 -6.12066931  1.291429  ]
 [ 0.         -7.06986253 -4.64043587  1.291429  ]
 [ 0.         -7.06986253 -3.16020275  1.291429  ]
 [ 0.         -7.06986253 -1.67996929  1.291429  ]]
jac: (500, 4)
[[5.2266026e+01 1.0952133e+00 2.0918984e+00 9.8287677e-03]]
(1000, 4)
jac: (1000, 4)
[[5.2266026e+01 1.0952133e+00 2.0918984e+00 9.8287677e-03]]
(1000, 4) [ 0.         -7.06986253 -7.60090246  1.29345603] [[ 0.         -7.06986253 -7.60090246  1.291429  ]
 [ 0.         -7.06986253 -7.60090246  1.349286  ]
 [ 0.         -7.06986253 -7.60090246  1.407143  ]
 [ 0.         -7.06986253 -7.60090246  1.465     ]
 [ 0.         -7.06986253 -7.60090246  1.522857  ]
 [ 0.         -7.06986253 -7.60090246  1.580714  ]
 [ 0.         -7.06986253 -7.60090246  1.638571  ]
 [ 0.         -7.06986253 -7.60090246  1.696429  ]] [[ 0.         -7.06986251 -7.60090241  1.29571462]
 [ 0. 

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

with torch.no_grad():
    preds = torch.exp(net_regr.module_(train_x)).cpu().numpy()
targets = torch.exp(train_y).cpu().numpy()
print(r2_score(targets, preds))
print(mean_absolute_error(targets, preds))
print(mean_squared_error(targets, preds))
print(mean_absolute_percentage_error(targets, preds))

with torch.no_grad():
    preds = torch.exp(net_regr.module_(test_x)).cpu().numpy()
targets = torch.exp(test_y).cpu().numpy()
print(r2_score(targets, preds))
print(mean_absolute_error(targets, preds))
print(mean_squared_error(targets, preds))
print(mean_absolute_percentage_error(targets, preds))

plt.scatter(targets, preds)
plt.axis('square')
plt.plot(np.linspace(-0.001, 0.007), np.linspace(-0.001, 0.007), 'r-')
plt.xlim(-0.001, 0.007)
plt.ylim(-0.001, 0.007)
plt.savefig("test_scatter.png", dpi=300)
plt.close()

In [5]:
allpred_X = np.vstack((pred_X[cho_idx,:],X1))
print(tmaty['ext'].shape, y1.shape)
testy     = np.append(tmaty['ext'],np.exp(y1[:,0]))
Xp = torch.Tensor(allpred_X).cuda()
Xp = (Xp-x_mean)/x_std
with torch.no_grad():
    pred_y = torch.exp(net_regr.module_(Xp)).cpu().numpy()

(101360,) (101360, 1)


In [6]:
from scatter_plot import scatter
fig = plt.figure(figsize=(6.5,4.5))
ax = fig.add_subplot(111)
ymi = np.nanmin(np.append(testy,pred_y))
yma = np.nanmax(np.append(testy,pred_y))
paths, slope, intercept = scatter(ax, testy, pred_y, color='b',label_p = 'lower right')
plt.xlim(ymi, yma)
plt.ylim(ymi, yma)
ax.set_xlabel(r'Tmatrix cext ($\mu$m$^{-1}$)')
ax.set_ylabel(r'NN Predict cext ($\mu$m$^{-1}$)')
plt.tight_layout()
plt.savefig("img/TMlogX_scatter_tanh_base_oriJ_fine_n.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()

R = 0.53 (p < 0.01)


In [9]:
from torch.utils.data import DataLoader, TensorDataset
pred_X_t = torch.tensor(allpred_X).float()
loader = DataLoader( pred_X_t, batch_size=5000)
all_yp = np.array([])
all_Jp = np.array([])
for data in loader:
    Xp = data.cuda()
    Xp = (Xp-x_mean)/x_std
    pred_y, pred_J, xt = get_ut_jacobian(net_regr.module_, Xp)
    #print(pred_y.shape, pred_J.shape)
    if len(all_yp) == 0:
        all_yp = pred_y
        all_Jp = pred_J
    else:    
        all_yp = np.vstack((all_yp, pred_y))
        all_Jp = np.vstack((all_Jp, pred_J))
    print(all_yp.shape, all_Jp.shape)

(5000, 1) (5000, 4)
(10000, 1) (10000, 4)
(15000, 1) (15000, 4)
(20000, 1) (20000, 4)
(25000, 1) (25000, 4)
(30000, 1) (30000, 4)
(35000, 1) (35000, 4)
(40000, 1) (40000, 4)
(45000, 1) (45000, 4)
(50000, 1) (50000, 4)
(55000, 1) (55000, 4)
(60000, 1) (60000, 4)
(65000, 1) (65000, 4)
(70000, 1) (70000, 4)
(75000, 1) (75000, 4)
(80000, 1) (80000, 4)
(85000, 1) (85000, 4)
(90000, 1) (90000, 4)
(95000, 1) (95000, 4)
(100000, 1) (100000, 4)
(105000, 1) (105000, 4)
(110000, 1) (110000, 4)
(115000, 1) (115000, 4)
(120000, 1) (120000, 4)
(125000, 1) (125000, 4)
(130000, 1) (130000, 4)
(135000, 1) (135000, 4)
(140000, 1) (140000, 4)
(145000, 1) (145000, 4)
(150000, 1) (150000, 4)
(155000, 1) (155000, 4)
(160000, 1) (160000, 4)
(165000, 1) (165000, 4)
(170000, 1) (170000, 4)
(175000, 1) (175000, 4)
(180000, 1) (180000, 4)
(185000, 1) (185000, 4)
(190000, 1) (190000, 4)
(195000, 1) (195000, 4)
(200000, 1) (200000, 4)
(202720, 1) (202720, 4)


In [10]:
Jlnk  = J1[:,0] * np.exp(X1[:,2])
Jn    = J1[:,1]
reJ   = np.hstack((Jlnk.reshape(-1,1), Jn.reshape(-1,1)))
testJ = np.vstack((tmatJ['ext'].T,reJ))

print(testJ.shape)
from scatter_plot import scatter
xx = ['lnk','n']
xx_use = {}
cb_t   = {}
xx_use['lnk'] = allpred_X[:,2]
xx_use['n']   = allpred_X[:,3]
cb_t['lnk']   = xx_use['lnk'][:181*14*5:181*14]
cb_t['n']     = np.sort(np.append(X1[::181*14*5,3], pred_X[cho_idx,3][::181*14*5]))
print(cb_t['lnk'], cb_t['n'])
for j in range(2):
    fig = plt.figure(figsize=(6.5,4.5))
    ax = fig.add_subplot(111)
    ymi = np.nanmin(np.append(testJ[:,j],all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j]))
    yma = np.nanmax(np.append(testJ[:,j],all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j]))
    print(len(cb_t[xx[j]]))
    paths, slope, intercept = scatter(ax, testJ[:,j], all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j], 
                                      color=xx_use[xx[j]],fig=fig,cbar_label=xx[j],cbar_ticks=cb_t[xx[j]],
                                      label_p = 'lower right')
    plt.xlim(ymi, yma)
    plt.ylim(ymi, yma)
    ax.set_xlabel(r'$\frac{\partial{cext}}{\partial{'+xx[j]+'}}$ from Tmatrix')
    ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{'+xx[j]+'}}$ from NN')
    plt.tight_layout()
    plt.savefig("img/TMlogX_Jac"+xx[j]+"_scatter_tanh_base_lnJ_fine_n.png", dpi=300, facecolor='w', edgecolor='w')
    plt.close()

(202720, 2)
[-7.60090246 -6.12066931 -4.64043587 -3.16020275 -1.67996929] [1.291429   1.29571462 1.30000031 1.304286   1.3085717  1.31285739
 1.31714296 1.32142866 1.32571435 1.349286   1.407143   1.465
 1.522857   1.580714   1.638571   1.696429  ]
5
cbar_tick [-7.60090246 -6.12066931 -4.64043587 -3.16020275 -1.67996929]
R = 0.78 (p < 0.01)
16
cbar_tick [1.291429   1.29571462 1.30000031 1.304286   1.3085717  1.31285739
 1.31714296 1.32142866 1.32571435 1.349286   1.407143   1.465
 1.522857   1.580714   1.638571   1.696429  ]
R = 0.04 (p < 0.01)


In [10]:
from scatter_plot import scatter
kk  = np.exp(allpred_X[:,2])
kname = ['{:.4f}'.format(ik) for ik in kk[:181*14*5:181*14]]
fig = plt.figure(figsize=(6.5,4.5))
ax = fig.add_subplot(111)
j = 0
ymi = np.nanmin(np.append(testJ[:,j]/kk,all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j]/kk))
yma = np.nanmax(np.append(testJ[:,j]/kk,all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j]/kk))
paths, slope, intercept = scatter(ax, testJ[:,j]/kk, all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j]/kk, 
                                  color=xx_use[xx[j]],fig=fig,cbar_label='k',cbar_ticks=cb_t[xx[j]],
                                  cticklabel=kname, label_p = 'lower right')
plt.xlim(ymi, yma)
plt.ylim(ymi, yma)
ax.set_xlabel(r'$\frac{\partial{cext}}{\partial{k}}$ from Tmatrix')
ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{k}}$ from NN')
plt.tight_layout()
plt.savefig("img/TMlogX_Jack_scatter_tanh_fine_n_0.1scale.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()
tJ  = tmatJ['ext'][j,:]/kk
pJ  = all_Jp[:,2+j]/x_std.cpu().numpy()[0][2+j]/kk
idx = np.where((tJ > 0.0088) & (tJ < 0.0089))
print(tJ[idx], pJ[idx])

cbar_tick [-7.60090246 -6.12066931 -4.64043587 -3.16020275 -1.67996929]
R = 0.08 (p < 0.01)


ValueError: operands could not be broadcast together with shapes (101360,) (202720,) 

In [ ]:
numk = 5
kk  = np.exp(pred_X[cho_idx,2])
for i in range(numk):
    fig = plt.figure(figsize=(5.5,4.5))
    ax = fig.add_subplot(111)
    j = 0
    idx2 = np.arange(181*14*i,181*14*(i+1))    
    ymi = np.nanmin(np.append(tmatJ['ext'][j,idx2]/kk[idx2],all_Jp[idx2,2+j]/x_std.cpu().numpy()[0][2+j]/kk[idx2]))
    yma = np.nanmax(np.append(tmatJ['ext'][j,idx2]/kk[idx2],all_Jp[idx2,2+j]/x_std.cpu().numpy()[0][2+j]/kk[idx2]))
    paths, slope, intercept = scatter(ax, tmatJ['ext'][j,idx2]/kk[idx2], all_Jp[idx2,2+j]/x_std.cpu().numpy()[0][2+j]/kk[idx2], 
                                  color='b',label_p = 'lower right')
    ax.set_title('k = {:.4f}'.format(kk[181*14*i]))
    plt.xlim(ymi, yma)
    plt.ylim(ymi, yma)
    ax.set_xlabel(r'$\frac{\partial{cext}}{\partial{k}}$ from Tmatrix')
    ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{k}}$ from NN')
    plt.tight_layout()
    plt.savefig("img/TMlogX_Jack_scatter_tanh0.01reg_k"+str(i)+".png", dpi=300, facecolor='w', edgecolor='w')
    plt.close()

In [ ]:
n = 4
inputx = pred_X[cho_idx[(n-1)*181*14:(n-1)*181*14+181],:]
Xp = torch.Tensor(inputx).cuda()
Xp = (Xp-x_mean)/x_std
pred_y, pred_J, xt = get_ut_jacobian(net_regr.module_, Xp)
fig = plt.figure(figsize=(6.5,4.5))
ax = fig.add_subplot(111)
tJ = tmatJ['ext'][0,(n-1)*181*14:(n-1)*181*14+181] / np.exp(inputx[:,2])
ax.scatter(inputx[:,0], tJ, label='Tmatrix' )
ax.scatter(inputx[:,0], pred_J[:,2]/x_std.cpu().numpy()[0][2]/np.exp(inputx[:,2]), label='NN' )
ax.set_xlabel('angle')
ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{k}}$')
plt.legend()
plt.tight_layout()
plt.savefig("img/TMlogX_Jack_tanh0.01reg_angle.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()
print(tJ)

In [ ]:
from scatter_plot import scatter
yp = []
xx = ['k','n']
aveJ = {}
aveJ['k'] = []
aveJ['n'] = []
for n in range(14*5*8):
    inputx = pred_X[cho_idx[n*181:(n+1)*181],:]
    Xp = torch.Tensor(inputx).cuda()
    Xp = (Xp-x_mean)/x_std
    pred_y, pred_J, xt = get_ut_jacobian(net_regr.module_, Xp)
    yp.append( np.nanmean(pred_y) )
    Jack = pred_J[:,2] / x_std.cpu().numpy()[0][2]/ np.exp(inputx[:,2])
    Jacn = pred_J[:,3] / x_std.cpu().numpy()[0][3]
    aveJ['k'].append( np.nanmean(Jack) )
    aveJ['n'].append( np.nanmean(Jacn) )

xx_use = {}
cb_t   = {}
xname  = {}
xx_use['k'] = pred_X[cho_idx[::181],2]
xx_use['n'] = pred_X[cho_idx[::181],3]
cb_t['k']   = xx_use['k'][:14*5:14]
cb_t['n']   = xx_use['n'][::14*5]
xname['k']  = ['{:.4f}'.format(np.exp(ik)) for ik in cb_t['k']]
xname['n']  = ['{:.4f}'.format(inn) for inn in cb_t['n']]
for j in range(len(xx)):
    ix = xx[j]
    fig = plt.figure(figsize=(6.5,4.5))
    ax = fig.add_subplot(111)
    if j == 0:
        tJ = tmatJ['ext'][j,::181] / np.exp(pred_X[cho_idx[::181],2])
    else:
        tJ = tmatJ['ext'][j,::181]
    ymi = np.nanmin(np.append(tJ,np.array(aveJ[ix])))
    yma = np.nanmax(np.append(tJ,np.array(aveJ[ix])))
    paths, slope, intercept = scatter(ax, tJ, np.array(aveJ[ix]), 
                                  color=xx_use[ix],fig=fig,cbar_label='k',cbar_ticks=cb_t[ix],
                                  cticklabel=xname[ix], label_p = 'lower right')
    plt.xlim(ymi, yma)
    plt.ylim(ymi, yma)
    ax.set_xlabel(r'$\frac{\partial{cext}}{\partial{'+ix+'}}$ from Tmatrix')
    ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{'+ix+'}}$ from NN')
    plt.tight_layout()
    plt.savefig("img/TMlogX_Jac"+ix+"_ave_tanh0.01reg_scatter.png", dpi=300, facecolor='w', edgecolor='w')
    plt.close()

In [ ]:
# 181,14,5,8
n = 1
j = 0
useX = pred_X[cho_idx,:]
inputx = useX[n::181,:]
Xp = torch.Tensor(inputx).cuda()
Xp = (Xp-x_mean)/x_std
pred_y, pred_J, xt = get_ut_jacobian(net_regr.module_, Xp)

fig = plt.figure(figsize=(5,4.5))
ax = fig.add_subplot(111)
tJ = tmatJ['ext'][j,n::181] / np.exp(inputx[:,2])
ax.scatter(np.exp(inputx[:14*5:14,2]), tJ[:14*5:14], label='Tmatrix' )
ax.scatter(np.exp(inputx[:14*5:14,2]), pred_J[:14*5:14,2]/x_std.cpu().numpy()[0][2]/np.exp(inputx[:14*5:14,2]), label='NN' )
ax.set_xlabel('k')
ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{k}}$')
plt.legend()
plt.tight_layout()
plt.savefig("img/TMlogX_Jack_k_tanh0.01reg.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()

fig = plt.figure(figsize=(5,4.5))
ax = fig.add_subplot(111)
tmy = tmaty['ext'][n::181] 
ax.plot(np.exp(inputx[:14*5:14,2]), tmy[:14*5:14], 'o-', label='Tmatrix' )
ax.plot(np.exp(inputx[:14*5:14,2]), pred_y[:14*5:14], 'o-', label='NN' )
ax.set_xlabel('k')
ax.set_ylabel('cext')
plt.legend()
plt.tight_layout()
plt.savefig("img/TMlogX_ext_k_tanh0.01reg.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()

fig = plt.figure(figsize=(5,4.5))
ax = fig.add_subplot(111)
ax.plot(inputx[::14*5,3], tmy[::14*5], 'o-', label='Tmatrix' )
ax.plot(inputx[::14*5,3], pred_y[::14*5], 'o-', label='NN' )
ax.set_xlabel('n')
ax.set_ylabel('cext')
plt.legend()
plt.tight_layout()
plt.savefig("img/TMlogX_ext_n_tanh0.01reg.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()

fig = plt.figure(figsize=(5,4.5))
ax = fig.add_subplot(111)
ax.plot(np.exp(inputx[:14,1]), tmy[:14], 'o-', label='Tmatrix' )
ax.plot(np.exp(inputx[:14,1]), pred_y[:14], 'o-', label='NN' )
ax.set_xlabel('r')
ax.set_ylabel('cext')
plt.legend()
plt.tight_layout()
plt.savefig("img/TMlogX_ext_r_tanh0.01reg.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()


In [ ]:
fig = plt.figure(figsize=(5,4.5))
ax = fig.add_subplot(111)
j  = 1
tJ = tmatJ['ext'][j,n::181]
ax.scatter(inputx[::14*5,3], tJ[::14*5], label='Tmatrix' )
ax.scatter(inputx[::14*5,3], pred_J[::14*5,3]/x_std.cpu().numpy()[0][3], label='NN' )
ax.set_xlabel('n')
ax.set_ylabel(r'$\frac{\partial{cext}}{\partial{n}}$')
plt.legend()
plt.tight_layout()
plt.savefig("img/TMlogX_Jacn_n_tanh0.01reg.png", dpi=300, facecolor='w', edgecolor='w')
plt.close()

In [10]:
import numpy as np
a = np.arange(24).reshape((2,3,4))
print(a)
print(np.sum(a, axis=0))

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
[[12 14 16 18]
 [20 22 24 26]
 [28 30 32 34]]
